In [26]:
import os
import numpy as np
import time

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img

import pandas as pd
import seaborn as sns

import csv

%matplotlib inline 

In [27]:
from sklearn.cluster import KMeans
import cv2

In [28]:
# base directory
base_dir = "/home/dmsai2/Desktop/DCC2022/data"
processed_dir = "/home/dmsai2/Desktop/DCC2022/pdata"
sub_data_dir = "/home/dmsai2/Desktop/DCC2022/tdata"

In [29]:
# load class list from directory
cls_list = os.listdir(base_dir)
print(*cls_list)

L2_21 L2_24 L2_15 L2_27 L2_30 L2_50 L2_40 L2_39 L2_20 L2_45 L2_3 L2_12 L2_41 L2_10 L2_34 L2_52 L2_33 L2_46 L2_44 L2_25


In [30]:
numdata = 0
numclass = 0
data_list = {}

for cls in cls_list:
    imlist = os.listdir(base_dir + "/" + cls)
    print(cls, end=' : ')
    print(len(imlist))
    numdata = numdata + len(imlist)
    data_list[cls] = imlist
    print("-"*12)

numclass = len(cls_list)

print("num of class :", numclass)
print("num of data :", numdata)

L2_21 : 410
------------
L2_24 : 782
------------
L2_15 : 1509
------------
L2_27 : 426
------------
L2_30 : 364
------------
L2_50 : 781
------------
L2_40 : 180
------------
L2_39 : 454
------------
L2_20 : 410
------------
L2_45 : 631
------------
L2_3 : 595
------------
L2_12 : 518
------------
L2_41 : 593
------------
L2_10 : 1893
------------
L2_34 : 419
------------
L2_52 : 382
------------
L2_33 : 6206
------------
L2_46 : 2214
------------
L2_44 : 547
------------
L2_25 : 6189
------------
num of class : 20
num of data : 25503


In [31]:
numsubcls = 0
subcls_list = {}

for cls in cls_list:
    sub_list = os.listdir(sub_data_dir + "/" + cls)
    subcls_list[cls] = sub_list
    
numsubcls = len(subcls_list)
print("num of subclass :", numsubcls)

num of subclass : 20


In [32]:
data = []
cnt = 0

if os.path.isfile('processed_df.csv'):
    print("reading data from csv.")
    df = pd.read_csv('processed_df.csv', index_col=0)
    df = df.astype({'img':'string', 'label':'string', 'ftype':'string', 'cmap':'string', 'fsize':'float', 'fmtime':'datetime64[ns]', 'width':'int', 'height':'int', 'channel':'int', 'subclass':'string', 'mode_rate':'float'})
    print(df.head())
    
else:
    print("generating data")
    for label in data_list.keys():
        for subcls in subcls_list[label]:
            subcls_path = f"{sub_data_dir}/{label}/{subcls}"
            imglist = os.listdir(subcls_path)

            for img_dir in imglist:
                img_path = f"{subcls_path}/{img_dir}"
                img = Image.open(img_path)
                imgarr = np.array(img)
                if len(imgarr.shape) == 2:
                    w, h = imgarr.shape
                    c = 1
                else:
                    w, h, c = imgarr.shape
                fsize = (os.path.getsize(img_path) / 1024.)
                fmtime = time.ctime(os.path.getmtime(img_path))
                data.append([img_dir[:-4], label, img_dir[-3:], img.mode, fsize, fmtime, w, h, c, subcls])
                cnt += 1
                print(f"{(int((cnt / numdata) * 100000.) / 1000.)}% - {cnt}/{numdata}")
    
    df = pd.DataFrame(data, columns=['img','label','ftype', 'cmap', 'fsize', 'fmtime', 'width', 'height', 'channel', 'subclass'])
    df = df.astype({'img':'string', 'label':'string', 'ftype':'string', 'cmap':'string', 'fsize':'float', 'fmtime':'datetime64[ns]', 'width':'int', 'height':'int', 'channel':'int', 'subclass':'string'})
    print(df.head())

reading data from csv.
                    img  label ftype cmap       fsize              fmtime  \
0  qfknrhrrdikepslumadh  L2_21   jpg  RGB   24.222656 2021-12-17 17:53:18   
1  ixxhosetjwabxnntumca  L2_21   jpg  RGB   55.888672 2021-12-17 17:53:24   
2  vyaroxjwnkljwbhkzphd  L2_21   jpg  RGB  129.636719 2021-12-17 17:53:22   
3  jiicqwbbzhvvjxhozaji  L2_21   jpg  RGB  375.979492 2021-12-17 17:53:06   
4  kfxkkclkrczktrfddomu  L2_21   jpg  RGB  115.896484 2021-12-17 17:53:24   

   width  height  channel subclass  mode_rate  
0    300     300        3     plug  94.719184  
1   1200    1200        3     plug  94.718980  
2    700     700        3     plug  94.719184  
3   2480    3508        3     plug  94.719184  
4   3508    2480        3     plug  94.647143  


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25503 entries, 0 to 25502
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   img        25503 non-null  string        
 1   label      25503 non-null  string        
 2   ftype      25503 non-null  string        
 3   cmap       25503 non-null  string        
 4   fsize      25503 non-null  float64       
 5   fmtime     25503 non-null  datetime64[ns]
 6   width      25503 non-null  int64         
 7   height     25503 non-null  int64         
 8   channel    25503 non-null  int64         
 9   subclass   25503 non-null  string        
 10  mode_rate  25503 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), string(5)
memory usage: 2.3 MB


In [34]:
df.groupby('ftype')['cmap'].value_counts()

ftype  cmap
jpg    RGB     23944
       CMYK       42
       L           6
png    RGBA     1127
       RGB       374
       P          10
Name: cmap, dtype: int64

In [35]:
def centroid_histogram(clt):
    # grab the number of different clusters and create a histogram
    # based on the number of pixels assigned to each cluster
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()

    # return the histogram
    return hist

def plot_colors(hist, centroids):
    # initialize the bar chart representing the relative frequency
    # of each of the colors
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0

    # loop over the percentage of each cluster and the color of
    # each cluster
    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                      color.astype("uint8").tolist(), -1)
        startX = endX

    # return the bar chart
    return bar

def analyzeimg(img_dir : str, isprint=False):
    img = Image.open(img_dir)
    img = img.convert('RGB')

    img = np.array(img)
    w, h, c = img.shape

    rate = min(w, h) / 200.
    rw, rh = int(w / rate), int(h / rate)
    
    img = cv2.resize(img, dsize=(rw, rh))
    
    w, h, c = img.shape
    
    img = img.reshape((w * h, 3))
    
    if isprint: plt.imshow(img)
    
    k = 30
    clt = KMeans(n_clusters=k)
    clt.fit(img)
    
    if isprint:
        for center in clt.cluster_centers_:
            print("color :", center)
        
    hist = centroid_histogram(clt)
    if isprint: print("each % :", hist)
    
    bar = plot_colors(hist, clt.cluster_centers_)

    if isprint:
        # show our color bart
        plt.figure()
        # plt.axis("off")
        plt.imshow(bar)
        plt.show()
    
    avg = sum([clt.cluster_centers_[i] * hist[i] for i in range(k)])
    mode = clt.cluster_centers_[list(hist).index(max(hist))]
    if isprint: print(f"average : {avg}\nmode : {mode} {int(hist[list(hist).index(max(hist))]*100)}%")
    
    # average, mode, clusters(RGB), hist(each rate), rate of mode color
    return avg, mode, clt.cluster_centers_, hist, hist[list(hist).index(max(hist))]*100

In [36]:
avg, mode, clusters, hist, mode_rate = analyzeimg(base_dir + "/L2_52/uehhbgclrdhrnrkuuenj.jpg", False)
mode_rate

30.167500000000004

In [37]:
df.describe()

,fsize,width,height,channel,mode_rate
count,25503.000000,25503.000000,25503.000000,25503.000000,25503.000000
mean,102.552891,1336.834216,1284.244089,3.044583,3.819827
std,123.861621,1166.400418,1069.300772,0.215317,18.629321
min,2.043945,97.000000,162.000000,1.000000,0.000000
25%,20.310059,375.000000,500.000000,3.000000,0.000000
50%,57.165039,700.000000,700.000000,3.000000,0.000000
75%,163.046387,2480.000000,2480.000000,3.000000,0.000000
max,3031.055664,5000.000000,5001.000000,4.000000,94.730816


In [39]:
for i, row in df.iterrows():
    print(f"{i}/{numdata} {i/numdata*100}%", row['label'], row['img'], end=' ')
    avg, mode, clusters, hist, mode_rate = analyzeimg(f"{base_dir}/{row['label']}/{row['img']}.{row['ftype']}", isprint=False)
    df.loc[i, 'mode_rate'] = mode_rate
    print("mode rate :", mode_rate)
    
print("done")

0/25503 0.0% L2_21 qfknrhrrdikepslumadh mode rate : 60.4275
1/25503 0.003921107320707368% L2_21 ixxhosetjwabxnntumca mode rate : 89.7975
2/25503 0.007842214641414736% L2_21 vyaroxjwnkljwbhkzphd mode rate : 74.25
3/25503 0.011763321962122103% L2_21 jiicqwbbzhvvjxhozaji mode rate : 76.74822695035462
4/25503 0.01568442928282947% L2_21 kfxkkclkrczktrfddomu mode rate : 96.46631205673759
5/25503 0.01960553660353684% L2_21 gtqnppjlorrxfsyvynko 

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


mode rate : 69.22
6/25503 0.023526643924244205% L2_21 xnywobdzwmldmdhaksrj mode rate : 85.08
7/25503 0.027447751244951572% L2_21 exnvonenhlvkwxilgepk mode rate : 83.52304964539007
8/25503 0.03136885856565894% L2_21 hkforrdlkwrebciozzls mode rate : 51.06
9/25503 0.03528996588636631% L2_21 zacluczxkpugvptwwklk mode rate : 86.06205673758865
10/25503 0.03921107320707368% L2_21 ktrcaqqdeokgrxblhfgg mode rate : 78.18971631205673
11/25503 0.04313218052778105% L2_21 pocgcvdqpawkpiuiinie mode rate : 72.27749999999999
12/25503 0.04705328784848841% L2_21 nklfppcjcwokqzqufhaz mode rate : 80.205
13/25503 0.050974395169195774% L2_21 wtzonodmnxncddrwiqeu 

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


mode rate : 71.08
14/25503 0.054895502489903145% L2_21 hetfwssaknofzdiptyay mode rate : 83.94
15/25503 0.058816609810610515% L2_21 tazsyjmycvnaidiywlfk 

KeyboardInterrupt: 

In [ ]:
df.to_csv('processed_df.csv', mode='w')